<a href="https://colab.research.google.com/github/pcpiscator/2T2021/blob/main/Previs%C3%A3o_Conluio_(Baseado_no_Titanic_).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Laboratório: Previsão Conluio (base Titanic )
Tentativa de Previsão de Conluio entre empresas em licitações.

# Previsão de Conluio entre empresas em licitações.

### Conteúdo Abordado:
1. Importar Bibliotecas Necessárias
2. Carregar e Explorar os Dados
3. Análise Exploratória
4. Visualização dos Dados
5. Limpeza dos Dados
6. Escolha do Melhor Modelo


## 1) Importar Bibliotecas Necessárias
Primeiramente, como sempre, importaremos bibliotecas Python como pandas, numpy, matplotlib e seaborn. Esta última biblioteca permite plotar uns gráficos diferentes, e mais bonitos se compararmos com a biblioteca matplotlib.

In [16]:
# bibliotecas para análise de dados
import numpy as np
import pandas as pd

# bibliotecas para visualização
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# configuração para ignorar mensagens de advertência (warning). 
# Muitas vezes estas mensagens atrapalham, pois pode trazer a informação de 
# algo está muito errado com nosso código quando na verdade temos apenas uma situação de advertência. 
import warnings
warnings.filterwarnings('ignore')

##2) Carregar e Explorar os Dados
Cabe lembrar que a pergunta que devemos responder para este desafio é a seguinte: **Quais grupos de empresas tiveram mais chances de participar de conluio em licitações? Para tanto,  deve-se explorar os dados das empresas (ex. CNPJ, Cód. da Licitação, Situação(vencedora ou perdedora etc.).** 

O *dataset* de licitações foi particionado em outros dois *datasets*: dados de treinamento e dados de teste. 
O *dataset* de treinamento contém os dados que serão utilizados para criar o nosso modelo preditivo, ou seja, os dados utilizados para **treinar o nosso modelo**. Por outro lado, o *dataset* de teste contém os dados que serão utilizados para testar e validar o desempenho do modelo com análise de sua acurácia e outras métricas. 

Neste passo você carregará os dois conjuntos de dados: treinamento e teste. Carregar neste contexto significa "importar" os dados dentro do nosso *notebook* utilizando o método `pd.read_csv`. Uma primeira olhada nos dados pode ser feita usando o método `describe()`.

In [18]:
# importar arquivos CSV: treinamento e teste. Note que os arquivos estão no Google Drive, e com isto, temos que acessá-los 
# através de um endereço web (URL) com caracteres 'estranhos' (chamados de 'hash'). Se os arquivos estivessem em seu computador,
# o acesso seria utilizando os nomes dos arquivos CSV treinamento.csv e teste.csv
#url_arquivo_treinamento_csv='https://drive.google.com/uc?export=view&id=1dJWrDlgp43l9qcZtSJ8ArZSWvxKWaQCx'
#url_arquivo_teste_csv='https://drive.google.com/uc?export=view&id=1wrYVj6EThqc1qX3QO_RWONGEQOf7tnJF'

treino = pd.read_csv('https://raw.githubusercontent.com/pcpiscator/2T2021/main/BaseLimpaTr.txt')
teste= pd.read_csv('https://raw.githubusercontent.com/pcpiscator/2T2021/main/BaseLimpaTs.txt')

# Dê uma olhada nos dados com o método describe. Passamos como parâmetro include=all para que todas as colunas do dataframe treinamento sejam apresentadas. 
treino.describe(include='all')
#teste.describe(include='all')

,licitacao,proponente,situacao
count,4.505600e+04,4.505600e+04,45055.000000
mean,2.923675e+06,1.940806e+13,0.700810
std,2.281637e+06,1.286377e+13,0.457908
min,8.190000e+02,2.937200e+10,0.000000
25%,1.014919e+06,8.923815e+12,0.000000
50%,2.308719e+06,1.810027e+13,1.000000
75%,4.448621e+06,2.774166e+13,1.000000
max,1.011352e+07,9.754595e+13,1.000000


## 3) Análise Exploratória
Agora, vamos analisar as variáveis (colunas) do *dataset* para verificar o tipo de dado, se dado faltante, se tem algum outro tipo de problema. Aqui resgatamos alguns dos conceitos de preparação de dados explorados nas semanas anteriores.
**Importante:** as variáveis (ou colunas) que utilizamos na criação da nossa solução, na forma de um modelo preditivo capaz de responder à nossa pergunta, recebem um nome especial em inglês, chamado de **features**. Uma feature é então uma variável que será utilizada como entrada na criação de um modelo preditivo.  

In [19]:
# mostra a lista de features (colunas) do dataframe 'treinamento'
print(treino.columns)

Index(['licitacao', 'proponente', ' situacao'], dtype='object')


In [20]:
# mostramos uma amostra do dataframe para termos uma idéia das variáveis. 
# o método 'sample' nos permite pegar neste caso uma amostra com 5 linhas
treino.sample(5)

,licitacao,proponente,situacao
33086,2152021,2620622000148,1.0
44937,9226121,12673901000123,1.0
2058,125119,17392053000106,0.0
1508,333519,29972807000178,1.0
24494,5223920,15114641000144,0.0


In [21]:
treino.head(5)

,licitacao,proponente,situacao
0,1527917,17886274000122,0.0
1,588618,5453608000276,1.0
2,2545418,198693000179,1.0
3,3612118,19786958000188,1.0
4,3612118,10642423000169,1.0


Podemos ter uma idéia inicial sobre os dados de passageiros. PassengerId é a identificação do passageiro a bordo. Temos o nome, masculino/feminino, idade, etc. Temos também o valor do ticket (Fare) em libra esterlina, a classe de cabine, a situação de embarque. 
Cabe observar que os nomes dos passageiros iniciam pelo sobrenome seguido de vírgula e o primeiro nome. A variável *Survived* é de muita importância para a nossa solução. Ele informa se um dado passageiro, com tais características, sobreviveu ou não ao naufrágio. Este variável é chamada de *variável-alvo* pois relaciona-se com a nossa pergunta chave: Quais grupos de pessoas tiveram mais chances de sobreviver neste desastre?

Abaixo colocamos uma tabela que pode vir ou não junto a um *dataset*. Esta tabela é chamada de *Dicionário de Dados*, e serve para descrever melhor cada variável. Se o dicionário de dados não for fornecido, cabe ao cientista de dados investigar junto ao cliente o significado de cada variável do conjunto de dados, por mais óbvia que o nome da variável possa parecer. 


<table> 

<td><b>Variável</b></td> <td> <b>Definição</b> </td> <td><b> Observação</b> </td><tr>
<td> survival	</td> <td> Sobrevivente	</td><td>0 = Não, 1 = Sim</td><tr>
<td>pclass</td>	<td>Classe do ticket</td>	<td>1 = Primeira, 2 = Segunda, 3 = Terceira</td><tr>
<td>sex</td>	<td>Sexo</td>	<td></td><tr>
<td>Age	</td><td>Idade em anos</td><td>	</td><tr>
<td>sibsp	</td><td>nro de irmãos/irmãs E esposa/esposo a bordo do Titanic</td><td>	</td><tr>
<td>parch	</td><td>nro de pais e filhos a bordo do Titanic </td><td>	<tr>
<td>ticket</td>	<td>Número do Ticket	</td> <td></td><tr>
<td>fare</td>	<td>Valor da passagem	</td> <td> </td><tr>
<td>cabin</td>	<td>nro da cabine</td><td></td>	<tr>
<td>embarked</td>	<td>Porto de Embarque</td>	<td>C = Cherbourg, Q = Queenstown, S = Southampton</td><tr>
</b>

</table>


Notas sobre as variáveis:
pclass: um indicador do status sócio-econômico do passageiro
1st = Upper (classe alta)
2nd = Middle (classe média)
3rd = Lower (classe baixa)

age: a idade é uma fração se o valor for menor que 1.  Se a idade é estimada, esta encontra-se no formato xx.5

sibsp: o dataset define relações familiares desta forma:
Sibling = irmão, irmã, "meio irmão", "meia irmã"
Spouse = esposa, esposo (namoradas e noivas foram ignoradas nos dados)

parch: o dataset define relações familiares desta forma...
Parent = mãe, pai
Child = filha, filho, enteada, enteado
Algumas crianças viajaram com suas babás, e com isto parch=0 nestes casos. 








#                          **Dicionário de Dados**
Abaixo colocamos uma tabela que pode vir ou não junto a um *dataset*. Esta tabela é chamada de *Dicionário de Dados*, e serve para descrever melhor cada variável. Se o dicionário de dados não for fornecido, cabe ao cientista de dados investigar junto ao cliente o significado de cada variável do conjunto de dados, por mais óbvia que o nome da variável possa parecer. 


<table> 

<td><b>Variável</b></td>  <td><b> Observação</b> </td><tr>
<td> licitacao-identificação da licitação	</td>
<tr><td> proponente - participante da licitação CNPJ	</td>
<tr><td>situacao</td>	<td>0 = perdedora, 1 = vencedora</td><tr>
</b>
</table>




* **Features Numéricas:** Age, Fare, SibSp, Parch
* **Features Categóricas:** Survived, Sex, Embarked, Pclass
* **Features Alfanuméricas (texto):** Ticket, Cabin

#### Quais são os tipos de dados para cada feature?
* Survived: int
* Pclass: int
* Name: string
* Sex: string
* Age: float
* SibSp: int
* Parch: int
* Ticket: string
* Fare: float
* Cabin: string
* Embarked: string

As *features* escolhidas devem ter propriedades que possam distinguir grupos de pessoas. Ou seja, variáveis muito específicas como por exemplo o nome do passageiro não servem neste nosso caso como uma *feature*. 


In [22]:
# visualizar um sumário do dataset de treinamento
treino.describe(include = "all")

,licitacao,proponente,situacao
count,4.505600e+04,4.505600e+04,45055.000000
mean,2.923675e+06,1.940806e+13,0.700810
std,2.281637e+06,1.286377e+13,0.457908
min,8.190000e+02,2.937200e+10,0.000000
25%,1.014919e+06,8.923815e+12,0.000000
50%,2.308719e+06,1.810027e+13,1.000000
75%,4.448621e+06,2.774166e+13,1.000000
max,1.011352e+07,9.754595e+13,1.000000


#### Algumas observações importantes (preparação de dados):
* Temos um total de 891 passageiros no nosso conjunto de treinamento (treinamento_df).
* Valores para a variável Age estão faltando em aproximadamente 19.8% dos casos.  Imagine que esta variável Age seja bem importante para a questão de sobrevivência neste desastre. Por isso, seria interessante preencher estas lacunas de dados faltantes. 
* A variável Cabin tem aproxidamente 77.1% dos seus dados faltando. Neste caso, por ser uma quantidade grande de dados faltantes, será difícil tentarmos preencher as lacunas vazias. Neste caso, uma estratégia (vista em semana anterior) é de remover estes valores do dataset. 
* A variável Embarked tem 0.22%  de valores faltantes, o que deve ser bem tranquilo e sem problemas maiores em nossa estratégia de preparação de dados. 


In [ ]:
# verifica se existe qualquer outro variável com dados faltantes
print(pd.isnull(treinamento_df).sum())

Podemos observar que exceto os valores faltantes mencionados acima, não existe mais valores NaN em outras variáveis. 

### Algumas previsões (de nossa cabeça)
* Sex: mulheres tem mais chance de sobrevivência. 
* SibSp/Parch: passageiros viajando sozinhos tem mais chance de sobrevivência.
* Age: crianças tem mais chance de sobrevivência.
* Pclass: pessoas de classe sócio-econômica mais alta tem chances de sobreviver. 


## 4) Visualização dos Dados
Agora podemos verificar se as nossas hipóteses acima se confirmam!


### Feature 'Sex'

In [ ]:
# plotar sobreviventes e gênero do passageiro. Observe que estamos usando a biblioteca seaborn para plotagem.
sns.barplot(x="Sex", y="Survived", data=treinamento_df)

# mostra percentuais de passageiros (feminino versus masculino) que sobreviveram
print("Percentual de passageiros gênero feminino que sobreviveram:", treinamento_df["Survived"][treinamento_df["Sex"] == 'female'].value_counts(normalize = True)[1]*100)

print("Percentual de passageiros gênero masculino que sobreviveram:", treinamento_df["Survived"][treinamento_df["Sex"] == 'male'].value_counts(normalize = True)[1]*100)

Como antecipado, passageira do gênero feminino teve chance maior de sobrivência se compararmos com o passageiro do gênero masculino. A feature Sex é essencial para novas previsões.


### Feature Pclass

In [ ]:
# plota sobreviventes por classe de cabine (Pclass)
sns.barplot(x="Pclass", y="Survived", data=treinamento_df)

# mostra percentual de pessoas por classe de cabine que sobreviveu
print("Percentual da Pclass = 1 que sobreviveu:", treinamento_df["Survived"][treinamento_df["Pclass"] == 1].value_counts(normalize = True)[1]*100)

print("Percentual de Pclass = 2 que sobreviveu:", treinamento_df["Survived"][treinamento_df["Pclass"] == 2].value_counts(normalize = True)[1]*100)

print("Percentual de Pclass = 3 que sobreviveu:", treinamento_df["Survived"][treinamento_df["Pclass"] == 3].value_counts(normalize = True)[1]*100)

Como antecipamos acima, pessoas de classe sócio-econômica mais alta teve maior taxa de sobrevivência neste naufrágio. (62.9% vs. 47.3% vs. 24.2%)

### Feature SibSp

In [ ]:
# plotar em gráficos de bar (barplot) a variável SibSp e as chances de sobrevivência
sns.barplot(x="SibSp", y="Survived", data=treinamento_df)

# imprimir os percentuais de até dois irmãos e/ou esposa/esposo
print("Percentual de SibSp = 0 que sobreviveu:", treinamento_df["Survived"][treinamento_df["SibSp"] == 0].value_counts(normalize = True)[1]*100)

print("Percentual de SibSp = 1 que sobreviveu:", treinamento_df["Survived"][treinamento_df["SibSp"] == 1].value_counts(normalize = True)[1]*100)

print("Percentual de SibSp = 2 que sobreviveu:", treinamento_df["Survived"][treinamento_df["SibSp"] == 2].value_counts(normalize = True)[1]*100)

Fica mais claro agora que pessoas com mais irmãos/irmãs incluindo esposa/esposo a bordo tiveram menores chances de sobrivivência. Entretanto, um fato relevante observado no gráfico de barras,  e contrário às expectativas, é que passsageiros com nenhum "irmão-irmã-esposo-esposa" tiveram menor chance de sobrevivência se compararmos com passageiros com um ou dois. (34.5% vs 53.4% vs. 46.4%)

### Feature Parch 

In [ ]:
# plotar um gráfico de barras para a variável Parch vs. Survived
sns.barplot(x="Parch", y="Survived", data=treinamento_df)
plt.show()

Passageiros com menos do que quatro pais ou crianças a bordo tiveram mais chances de sobrevivência comparando com os passageiros com quatro ou mais. Novamente, passageiros viajando sozinhos apresentaram menores chances de sobevivência. 



### Feature Age

In [ ]:
# ordena os valores de idade em categorias lógicas
treinamento_df["Age"] = treinamento_df["Age"].fillna(-0.5)      # esse preenchimento de valores foi visto nas semanas anteriores
teste_df["Age"] = teste_df["Age"].fillna(-0.5)
bins = [-1, 0, 5, 12, 18, 24, 35, 60, np.inf]
labels = ['Unknown', 'Baby', 'Child', 'Teenager', 'Student', 'Young Adult', 'Adult', 'Senior']
treinamento_df['AgeGroup'] = pd.cut(treinamento_df["Age"], bins, labels = labels)
teste_df['AgeGroup'] = pd.cut(teste_df["Age"], bins, labels = labels)

# plota gráfico de barras (Age vs. survival)
sns.barplot(x="AgeGroup", y="Survived", data=treinamento_df)
plt.show()

Bebês tiveram as maiores de chance de sobrevivência de qualquer dos grupos. 

<img src="https://upload.wikimedia.org/wikipedia/commons/a/a2/Titanic_lifeboat.jpg">

### Feature Cabin
A idéia é verificar se passageiros com números de cabines registrados são de classe social mais alta, e por isso com maiores chances de sobrevivência. 


In [ ]:
treinamento_df["CabinBool"] = (treinamento_df["Cabin"].notnull().astype('int'))
teste_df["CabinBool"] = (teste_df["Cabin"].notnull().astype('int'))

# calculamos aqui os percentuais de CabinBool vs. survived
print("Percentual de CabinBool = 1 que sobreviveu:", treinamento_df["Survived"][treinamento_df["CabinBool"] == 1].value_counts(normalize = True)[1]*100)

print("Percentual de CabinBool = 0 que sobreviveu:", treinamento_df["Survived"][treinamento_df["CabinBool"] == 0].value_counts(normalize = True)[1]*100)
#draw a bar plot of CabinBool vs. survival
sns.barplot(x="CabinBool", y="Survived", data=treinamento_df)
plt.show()

Em resumo, passageiros com valor de cabin registrado no dataset, tem maiores chances de sobrevivência (66.6% vs 29.9%)

É importante salientar que todas as análises acima são puramente baseadas em inspeções dos dados históricos. Ou seja, refletem estatísticas sobre dados e eventos que realmente aconteceram. 


<img src="https://www.euractiv.com/wp-content/uploads/sites/2/2016/02/titanic_orchestra_the_movie.jpeg" width = 480>

Orquestra no filme *Titanic* de James Cameron (1998). Isso mostra um pouco da falta de opções que os passageiros tinham e também a baixa expectativa de sobrevivência pela falta de barcos salva-vidas, e também de coletes salva-vidas. 

## 5) Limpeza dos Dados
Momento para limparmos nossos dados para tratar dados faltantes e informação desnecessária. 


### Inspecionando os dados de teste
Cabe lembrar que temos dois conjuntos de dados: treinamento (utilizado para criar modelos), e conjunto de dados para utilizado para validar o modelo criado.  
Vamos olhar o conjunto de teste. 

In [ ]:
teste_df.describe(include="all")


* Temos um total de 418 passageiros. 
* 1 valor para a variável Fare (preço do ticket) está faltando. 
* Em torno de 20.5% dos valores estão faltando para a varíavel Age. Precisamos preencher estas lacunas. 


### Feature Cabin

In [ ]:
# iniciamos removendo (dropping) valores da variável Cabin pois
# não podemos extrair algo muito útil desta variável
treinamento_df = treinamento_df.drop(['Cabin'], axis = 1)
teste_df = teste_df.drop(['Cabin'], axis = 1)

### Feature Ticket 

In [ ]:
# podemos também remover a variável Ticket pois esta não deve acrescentar muita coisa útil. O Ticket é um número, e dificilmente, pode ser 
# aproveitado para identificar um grupo de passageiros. O número do Ticket pode identificar sim um passageiro em específico, mas não 
# consegue conter informação acerca de um grupo de pessoas. 
treinamento_df = treinamento_df.drop(['Ticket'], axis = 1)
teste_Df = teste_df.drop(['Ticket'], axis = 1)

###  Feature Embarked

In [ ]:
# precisamos preencher os dados faltantes na variável Embarked com valores apropriados
print("Número de pessoas que embarcaram em Southampton (S):")
southampton = treinamento_df[treinamento_df["Embarked"] == "S"].shape[0]
print(southampton)

print("Número de pessoas que embarcaram em Cherbourg (C):")
cherbourg = treinamento_df[treinamento_df["Embarked"] == "C"].shape[0]
print(cherbourg)

print("Número de pessoas que embarcaram em Queenstown (Q):")
queenstown = treinamento_df[treinamento_df["Embarked"] == "Q"].shape[0]
print(queenstown)

Claramente a maioria das pessoas embarcou em Southampton (S). Podemos então preencher os dados faltantes na variável Embarked com o valor (S) de Southampton. Entenderam essa estratégia de preenchimento de valores faltantes? Utiliza a categoria com maior número de valores para então preencher os dados faltantes. 
 

In [ ]:
# preenchendo os dados faltantes na variável Embarked com o valor S
treinamento_df = treinamento_df.fillna({"Embarked": "S"})

### Feature Age


Vamos agora tratar os valores faltantes da variável Age. Partindo do ponto que um grande percentual dos valores estão faltando, seria ilógico preenchermos todos estes valores com o mesmo valor (da forma como fizemos com a variável Embarked). Vamos explorar uma outra alternativa para preencher os valores nulos. 

In [ ]:
# criamos um dataset 'combinado' que junta os nosos dois datasets (treinamento e teste)
combinado_df = [treinamento_df, teste_df]

# extraímos um Título para cada nome nos datasets treinamento_df e teste_df
for dataset in combinado_df:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

pd.crosstab(treinamento_df['Title'], treinamento_df['Sex'])

In [ ]:
# troca vários títulos de nomes por nomes mais comuns
for dataset in combinado_df:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Capt', 'Col',
    'Don', 'Dr', 'Major', 'Rev', 'Jonkheer', 'Dona'], 'Rare')
    
    dataset['Title'] = dataset['Title'].replace(['Countess', 'Lady', 'Sir'], 'Royal')
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

treinamento_df[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

In [ ]:
# faz o mapeamento de um dos grupos de títulos em um valor númerico
mapeamento_titulo = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Royal": 5, "Rare": 6}
for dataset in combinado_df:
    dataset['Title'] = dataset['Title'].map(mapeamento_titulo)
    dataset['Title'] = dataset['Title'].fillna(0)

treinamento_df.head()

Vamos tentar preencher os valores nulos na variável Age utilizando as idades mais comnuns para os títulos. 

In [ ]:
# preenchendo valores faltantes de idade com o grupo de idade para cada título
mr_age = treinamento_df[treinamento_df["Title"] == 1]["AgeGroup"].mode() #Adulto jovem
miss_age = treinamento_df[treinamento_df["Title"] == 2]["AgeGroup"].mode() #Estudante
mrs_age = treinamento_df[treinamento_df["Title"] == 3]["AgeGroup"].mode() #Adulto
master_age = treinamento_df[treinamento_df["Title"] == 4]["AgeGroup"].mode() #Bebê
royal_age = treinamento_df[treinamento_df["Title"] == 5]["AgeGroup"].mode() #Adulto
rare_age = treinamento_df[treinamento_df["Title"] == 6]["AgeGroup"].mode() #Adulto

idade_titulo_mapeamento = {1: "Young Adult", 2: "Student", 3: "Adult", 4: "Baby", 5: "Adult", 6: "Adult"}

#I tried to get this code to work with using .map(), but couldn't.
#I've put down a less elegant, temporary solution for now.
#train = train.fillna({"Age": train["Title"].map(age_title_mapping)})
#test = test.fillna({"Age": test["Title"].map(age_title_mapping)})

for x in range(len(treinamento_df["AgeGroup"])):
    if treinamento_df["AgeGroup"][x] == "Unknown":
        treinamento_df["AgeGroup"][x] = idade_titulo_mapeamento[treinamento_df["Title"][x]]
        
for x in range(len(teste_df["AgeGroup"])):
    if teste_df["AgeGroup"][x] == "Unknown":
        teste_df["AgeGroup"][x] = idade_titulo_mapeamento[teste_df["Title"][x]]

Agora que já preenchemos os valores faltantes de forma mais precisa (melhorias ainda precisam ser feitas na metodologia), é o momento de mapear cada grupo de idade para um valor numérico. 

In [ ]:
# mapeia cada valor de Idade a um valor numérico
mapeamento_idade = {'Baby': 1, 'Child': 2, 'Teenager': 3, 'Student': 4, 'Young Adult': 5, 'Adult': 6, 'Senior': 7}
treinamento_df['AgeGroup'] = treinamento_df['AgeGroup'].map(mapeamento_idade)
teste_df['AgeGroup'] = teste_df['AgeGroup'].map(mapeamento_idade)

#dropping the Age feature for now, might change
treinamento_df = treinamento_df.drop(['Age'], axis = 1)
teste_df = teste_df.drop(['Age'], axis = 1)

treinamento_df.head()

### Feature Name
We can drop the name feature now that we've extracted the titles.

In [ ]:
#drop the name feature since it contains no more useful information.
train = train.drop(['Name'], axis = 1)
test = test.drop(['Name'], axis = 1)

### Feature Sex 

In [ ]:
# mapeia cada valor da variável Sex para um valor numérico. value to a numerical value
# este tipo de preparação de dados é bem comum, ou seja, transformamos os valores categóricos texto em valore numéricos. 
# neste caso da variável Sex, temos "male" torna-se valor 0, e "female" torna-se valor 1
mapeamento_sex = {"male": 0, "female": 1}
treinamento_df['Sex'] = treinamento_df['Sex'].map(mapeamento_sex)
teste_df['Sex'] = teste_df['Sex'].map(mapeamento_sex)

treinamento_df.head()

###  Feature Embarked

In [ ]:
# mapeia cada valor da variável Embarked em um valor numérico
mapeamento_embarked = {"S": 1, "C": 2, "Q": 3}
treinamento_df['Embarked'] = treinamento_df['Embarked'].map(mapeamento_embarked)
teste_df['Embarked'] = teste_df['Embarked'].map(mapeamento_embarked)

treinamento_df.head()

###  Feature Fare
Agora podemos separar os valores da variável Fare (tarifa do ticket) em alguns grupos lógicos. 

It's time separate the fare values into some logical groups as well as filling in the single missing value in the test dataset.

In [ ]:
# preencher os valores faltantes na variável Fare no dataset de teste utilizando a média de tarifas para cada classe Pclass 
for x in range(len(teste_df["Fare"])):
    if pd.isnull(teste_df["Fare"][x]):
        pclass = teste_df["Pclass"][x] #Pclass = 3
        test["Fare"][x] = round(treinamento_df[treinamento_df["Pclass"] == pclass]["Fare"].mean(), 4)
        
# mapeia valores de Fare em grupos de valores numéricos
treinamento_df['FareBand'] = pd.qcut(treinamento_df['Fare'], 4, labels = [1, 2, 3, 4])
teste_df['FareBand'] = pd.qcut(teste_df['Fare'], 4, labels = [1, 2, 3, 4])

# remove valores da variável Fare
treinamento_df = treinamento_df.drop(['Fare'], axis = 1)
teste_df = teste_df.drop(['Fare'], axis = 1)

In [ ]:
# verifica dados de treinamento
treinamento_df.head()

In [ ]:
# verifica dados de teste
teste_df.head()

## 6) Escolha do Melhor Modelo

Como todo o bom projeto de ciência de dados até este ponto completamos em torno de 80% do cronograma. A partir deste momento, entramos na parte que todo o cientista de dados gostaria de iniciar, que é a criação e validação de modelos. 

Mas como observaram, a quantidade de operações de tratamento de dados, com limpeza, preenchimento de dados faltantes, e outros é muito grande, e pode comprometer o cronograma das atividades. 

A idéia é terminarmos a parte da modelagem na próxima semana, e com isto, deixando a expectativa da criação do nosso modelo preditivo (sim, utilizando aprendizado de máquina ou *machine learning*).

Continue tunado, semana que vem tem mais! 

Terminamos com a cena mais famosa do filme Titanic de 1998 dirigido por James Cameron, com Kate Winslet como Rose e Leonardo DiCaprio como Jack. Apesar de inspirado em alguns fatos reais, o filme conta a estória de Rose e Jack que se conhecem a bordo do Titanic. Os dois personagens vem de classes sociais diferentes, e os nossos dados reais apresenta muito forte este aspecto da classe sócio-econômica. Será que Rose e Jack constam na lista de nomes de passageiros reais do Titanic? Tente verificar isso com uma procura nos dados. 


<img src="https://www.gannett-cdn.com/-mm-/ae6c7665fa69ac7eec8a0d3a5dd5b6d77eb27b98/c=0-72-1721-1044/local/-/media/2017/12/19/WIGroup/Milwaukee/636492789022088927-TITANIC20P2.JPG?width=660&height=373&fit=crop&format=pjpg&auto=webp"> 

## Fontes:
* Este notebook é uma adaptação e tradução do notebook original disponibilizado por Nadim Tamer 
[Titanic Survival Predictions (Beginner)](https://www.kaggle.com/nadintamer/titanic-survival-predictions-beginner)